In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import seaborn as sns
from datetime import datetime
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/Shared drives/bds-final-project/scooter_data_clean.csv', index_col=0)
merge_data = pd.read_csv('/content/drive/Shared drives/bds-final-project/merged_data.csv', index_col=0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
data['date_of_month'] = [datetime.utcfromtimestamp(date).strftime('%d') for date in data["start_time"]]
data.head()

,trip_duration,trip_distance,start_time,end_time,month,hour,day_of_week,district_start,district_end,year,tract_start,tract_end,vehicle_type_bicycle,vehicle_type_moped,vehicle_type_scooter,date_of_month
0,420,869.0,1523390400,1523390400,4,20,2,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1,10
1,371,1037.0,1523389500,1523390400,4,19,2,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1,10
2,1687,4491.0,1524489300,1524491100,4,13,1,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1,23
3,524,633.0,1524865500,1524866400,4,21,5,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1,27
4,322,1114.0,1524865500,1524865500,4,21,5,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1,27


In [5]:
month_dict = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May' ,6: 'Jun', 7: 'Jul',8:'Aug' ,9: 'Sept', 10: 'Oct', 11: 'Nov',12: 'Dec'}
weekday_dict = {0: 'Sun', 1: 'Mon', 2: 'Tues', 3: 'Wed', 4: 'Thur', 5:'Fri', 6: 'Sat'}

merge_data = pd.read_csv('/content/drive/Shared drives/bds-final-project/merged_data.csv', index_col=0)
merge_data["weekday"] = [weekday_dict[num] for num in merge_data.day_of_week]
merge_data["month"] = [month_dict[datetime.strptime(strDate, '%Y-%m-%d').month] for strDate in merge_data.date]
merge_data["year"] = [datetime.strptime(strDate, '%Y-%m-%d').year for strDate in merge_data.date]
merge_data["date_of_month"] = [datetime.strptime(strDate, '%Y-%m-%d').day for strDate in merge_data.date]
merge_data["num_trips"] = merge_data.num_bicycle_trips + merge_data.num_moped_trips + merge_data.num_scooter_trips
merge_data["prev_day_trips"] = merge_data["num_trips"].shift(periods=1, fill_value=0)
merge_data = merge_data.drop(["date", "day_of_week", "num_bicycle_trips", "num_moped_trips", "num_scooter_trips"], axis=1)
merge_data.head()

,avg_trip_duration,avg_trip_distance,Max_Temperature_F,Avg_Temperature_F,Min_Temperature_F,Max_DewPoint_F,Avg_DewPoint_F,Min_DewPoint_F,Max_Humidity_%,Avg_Humidity_%,Min_Humidity_%,Max_WindSpeed_mph,Avg_WindSpeed_mph,Min_WindSpeed_mph,Max_Pressure_Hg,Avg_Pressure_Hg,Min_Pressure_Hg,Precipitation_inches,weekday,month,year,date_of_month,num_trips,prev_day_trips
0,943.000000,419.000000,80,72.7,65,71,64.4,40,90,76.6,40,24,12.1,0,29.5,29.2,29.1,0.00,Tues,Apr,2018,3,1,0
1,1360.333333,5691.333333,70,59.8,50,49,35.1,30,80,41.6,26,28,13.5,0,29.7,29.6,29.5,0.00,Wed,Apr,2018,4,3,1
2,1198.633929,2238.776786,78,68.0,56,67,57.6,47,87,70.0,56,16,9.0,0,29.6,29.4,29.3,0.00,Thur,Apr,2018,5,112,3
3,1118.914530,2616.518519,85,75.6,71,71,68.7,67,93,80.1,58,22,12.4,3,29.3,29.2,29.1,0.00,Fri,Apr,2018,6,351,112
4,970.707207,1815.882883,72,50.0,41,69,44.6,35,96,81.6,73,28,18.4,12,29.5,29.4,29.2,0.31,Sat,Apr,2018,7,222,351


In [6]:
###NOTE: Testing for report purposes. You can COMMENT and UNCOMMENT this line to utilize weather data

#merge_data = merge_data[["weekday", "month", "year", "date_of_month", "num_trips", "prev_day_trips", "avg_trip_duration", "avg_trip_distance"]]
#merge_data = merge_data[["weekday", "month", "year", "date_of_month", "num_trips", "prev_day_trips", "avg_trip_duration", "avg_trip_distance", "Avg_Temperature_F"]]
sum(merge_data["num_trips"])/len(merge_data)

6819.658620689655

In [7]:
weekday_dums = pd.get_dummies(merge_data["weekday"], prefix="day_")
month_dums = pd.get_dummies(merge_data["month"], prefix="mo_")
merge_data = pd.concat((merge_data, weekday_dums, month_dums), axis=1).drop(["weekday", "month"], axis=1)
merge_data.head()

,avg_trip_duration,avg_trip_distance,Max_Temperature_F,Avg_Temperature_F,Min_Temperature_F,Max_DewPoint_F,Avg_DewPoint_F,Min_DewPoint_F,Max_Humidity_%,Avg_Humidity_%,Min_Humidity_%,Max_WindSpeed_mph,Avg_WindSpeed_mph,Min_WindSpeed_mph,Max_Pressure_Hg,Avg_Pressure_Hg,Min_Pressure_Hg,Precipitation_inches,year,date_of_month,num_trips,prev_day_trips,day__Fri,day__Mon,day__Sat,day__Sun,day__Thur,day__Tues,day__Wed,mo__Apr,mo__Aug,mo__Dec,mo__Feb,mo__Jan,mo__Jul,mo__Jun,mo__Mar,mo__May,mo__Nov,mo__Oct,mo__Sept
0,943.000000,419.000000,80,72.7,65,71,64.4,40,90,76.6,40,24,12.1,0,29.5,29.2,29.1,0.00,2018,3,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
1,1360.333333,5691.333333,70,59.8,50,49,35.1,30,80,41.6,26,28,13.5,0,29.7,29.6,29.5,0.00,2018,4,3,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
2,1198.633929,2238.776786,78,68.0,56,67,57.6,47,87,70.0,56,16,9.0,0,29.6,29.4,29.3,0.00,2018,5,112,3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,1118.914530,2616.518519,85,75.6,71,71,68.7,67,93,80.1,58,22,12.4,3,29.3,29.2,29.1,0.00,2018,6,351,112,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,970.707207,1815.882883,72,50.0,41,69,44.6,35,96,81.6,73,28,18.4,12,29.5,29.4,29.2,0.31,2018,7,222,351,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso

mergeX = merge_data.drop(["num_trips", "avg_trip_duration", "avg_trip_distance"], axis=1)
mergeY = merge_data["num_trips"]
xTrain, xTest, yTrain, yTest = train_test_split(mergeX, mergeY, test_size = .30, random_state=42)

In [9]:
#note: adding in a lag of the previous day's trip count was VERY helpful, MSE went from ~52M to ~25M
regressor = LinearRegression().fit(xTrain, yTrain)
print("Train MSE: ", mean_squared_error(yTrain, regressor.predict(xTrain)))
print("Test MSE:  ", mean_squared_error(yTest, regressor.predict(xTest)))
print("Test MAE:  ", mean_absolute_error(yTest, regressor.predict(xTest)))

Train MSE:  22812295.272823997
Test MSE:   25266191.530140836
Test MAE:   3771.150176520922


Above output using all weather data:

*   Train MSE:  22,812,295.272823997 
*   Test MSE:   25,266,191.530140836
*   Test MAE:   3,771.150176520922

Above output using no weather data:
*   Train MSE:  23,798,786.280165028
*   Test MSE:   24,339,304.335490588
*   Test MAE:   3,648.2655441582515

Using only average temperature
Train MSE:  23753901.401045088
Test MSE:   24237005.647708174
Test MAE:   3647.7563410562334


In [10]:
for i in range(len(regressor.coef_)):
  print(xTrain.columns[i] + ": " + str(regressor.coef_[i]))

Max_Temperature_F: -98.62170127927739
Avg_Temperature_F: -182.05342170605329
Min_Temperature_F: -88.7061615345335
Max_DewPoint_F: -17.724972091687444
Avg_DewPoint_F: 398.4457187202553
Min_DewPoint_F: 8.587545222553748
Max_Humidity_%: -12.74778683065539
Avg_Humidity_%: -148.02389986390997
Min_Humidity_%: -63.672389262900715
Max_WindSpeed_mph: 74.40614167329103
Avg_WindSpeed_mph: -191.07333051247022
Min_WindSpeed_mph: 4.0670216252780405
Max_Pressure_Hg: -549.2305147225235
Avg_Pressure_Hg: -3147.6591744861303
Min_Pressure_Hg: 457.2346435874515
Precipitation_inches: 1543.8400509203825
year: -2053.746755706957
date_of_month: 2.96449565877685
prev_day_trips: 0.6655747479204818
day__Fri: 1550.4379391775676
day__Mon: -1135.9553744280054
day__Sat: 1480.1169970048522
day__Sun: -1931.25073632395
day__Thur: 416.6132017911616
day__Tues: -355.42613986177025
day__Wed: -24.535887359861007
mo__Apr: -883.9513088213436
mo__Aug: -372.4282934633772
mo__Dec: -1253.918994416428
mo__Feb: 1093.612454641705
mo_

In [11]:
#next, hyperparameter tune the regression and then re-report outcomes
from sklearn.model_selection import GridSearchCV

params = {
    'l1_ratio':[0,.2,.4,.6,.8,1],
    'random_state':[42],
    'alpha':[.25, .5, .75, 1, 5],
    'max_iter':[10000]
}
clf = ElasticNet()
grid = GridSearchCV(clf, params, verbose=3).fit(xTrain,yTrain)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] alpha=0.25, l1_ratio=0, max_iter=10000, random_state=42 .........


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10276974369.759796, tolerance: 5095063.584411824
  positive)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  alpha=0.25, l1_ratio=0, max_iter=10000, random_state=42, score=0.675, total=   0.4s
[CV] alpha=0.25, l1_ratio=0, max_iter=10000, random_state=42 .........


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10248707861.212292, tolerance: 5422651.369414163
  positive)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s


[CV]  alpha=0.25, l1_ratio=0, max_iter=10000, random_state=42, score=0.583, total=   0.4s
[CV] alpha=0.25, l1_ratio=0, max_iter=10000, random_state=42 .........


...

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11188779391.493692, tolerance: 5509039.273388546
  positive)


[CV]  alpha=5, l1_ratio=0, max_iter=10000, random_state=42, score=0.582, total=   0.4s
[CV] alpha=5, l1_ratio=0, max_iter=10000, random_state=42 ............


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10058684206.682718, tolerance: 5140903.467703449
  positive)


[CV]  alpha=5, l1_ratio=0, max_iter=10000, random_state=42, score=0.536, total=   0.4s
[CV] alpha=5, l1_ratio=0.2, max_iter=10000, random_state=42 ..........
[CV]  alpha=5, l1_ratio=0.2, max_iter=10000, random_state=42, score=0.678, total=   0.0s
[CV] alpha=5, l1_ratio=0.2, max_iter=10000, random_state=42 ..........
[CV]  alpha=5, l1_ratio=0.2, max_iter=10000, random_state=42, score=0.547, total=   0.0s
[CV] alpha=5, l1_ratio=0.2, max_iter=10000, random_state=42 ..........
[CV]  alpha=5, l1_ratio=0.2, max_iter=10000, random_state=42, score=0.561, total=   0.0s
[CV] alpha=5, l1_ratio=0.2, max_iter=10000, random_state=42 ..........
[CV]  alpha=5, l1_ratio=0.2, max_iter=10000, random_state=42, score=0.582, total=   0.0s
[CV] alpha=5, l1_ratio=0.2, max_iter=10000, random_state=42 ..........
[CV]  alpha=5, l1_ratio=0.2, max_iter=10000, random_state=42, score=0.537, total=   0.0s
[CV] alpha=5, l1_ratio=0.4, max_iter=10000, random_state=42 ..........
[CV]  alpha=5, l1_ratio=0.4, max_iter=1000

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:   19.5s finished


In [12]:
grid.best_params_

{'alpha': 1, 'l1_ratio': 1, 'max_iter': 10000, 'random_state': 42}

In [13]:
#We see a very slight improvement by implementing lasso, but not anything significant
elastic = ElasticNet(alpha=5, l1_ratio=1, max_iter=10000, random_state=42).fit(xTrain,yTrain)
#elastic = Lasso(alpha=40, max_iter=10000, random_state=42).fit(xTrain,yTrain)
print("Train MSE: ", mean_squared_error(yTrain, elastic.predict(xTrain)))
print("Test MSE:  ", mean_squared_error(yTest, elastic.predict(xTest)))
print("Test MAE:  ", mean_absolute_error(yTest, elastic.predict(xTest)))

Train MSE:  22824595.816311292
Test MSE:   25192882.821143743
Test MAE:   3753.906150590701


This is using the non-merged data. Please note following code only for reference and review purposes, not complete analysis

In [14]:
grouped_data = data.groupby(['year','month', 'date_of_month']).agg({'vehicle_type_bicycle':['sum'],'vehicle_type_moped':['sum'],'vehicle_type_scooter':['sum'],
                                                                    'day_of_week':['mean'], 'hour':['mean'],'trip_duration':['sum'], 'trip_distance':['mean'],
                                                                    'district_start':['count']}).reset_index()
grouped_data.columns = ['year', 'month', 'date_of_month', 'num_bicycle', 'num_moped', 'num_scooter', 'weekday', 'average_trip_hour', 'total_duration', 'mean_distance', 'num_trips']

grouped_data.head(3)

,year,month,date_of_month,num_bicycle,num_moped,num_scooter,weekday,average_trip_hour,total_duration,mean_distance,num_trips
0,2018,4,03,0,0,1,2.0,18.000000,943,419.000000,1
1,2018,4,04,0,0,3,3.0,14.000000,4081,5691.333333,3
2,2018,4,05,0,0,112,4.0,15.276786,134247,2238.776786,112


In [15]:
  month_dict = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May' ,6: 'Jun', 7: 'Jul',8:'Aug' ,9: 'Sept', 10: 'Oct', 11: 'Nov',12: 'Dec'}
  weekday_dict = {0: 'Sun', 1: 'Mon', 2: 'Tues', 3: 'Wed', 4: 'Thur', 5:'Fri', 6: 'Sat'}
    #Katherine - might be helpful to run a catboost and bring this back in as a categorical variable?
  #grouped_data['weekday'] = [weekday_dict[round(day)] for day in grouped_data['weekday']]
  grouped_data.head(1)

,year,month,date_of_month,num_bicycle,num_moped,num_scooter,weekday,average_trip_hour,total_duration,mean_distance,num_trips
0,2018,4,03,0,0,1,2.0,18.0,943,419.0,1


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression

train, test = train_test_split(grouped_data, test_size=.30)

In [17]:
def reportRegression(target_name, train, test):
  if target_name == "num_trips":
    LR = LinearRegression().fit(train.drop([target_name, "num_bicycle", "num_moped", "num_scooter"], axis=1), train[target_name])
    print("Predicting for: " + target_name)
    print("MSE: ", mean_squared_error(test[target_name], LR.predict(test.drop([target_name, "num_bicycle", "num_moped", "num_scooter"], axis=1))))
    print(LR.coef_)
  else:
    LR = LinearRegression().fit(train.drop([target_name], axis=1), train[target_name])
    print("Predicting for: " + target_name)
    print("MSE: ", mean_squared_error(test[target_name], LR.predict(test.drop([target_name], axis=1))))   

In [18]:
# Target variables - number of trips per day, average trip length, total duration of all trips.
for target in ["num_trips", "mean_distance", "total_duration"]:
  reportRegression(target, train, test)

  #Looks like we can get realistic predictions on number of trips, just have to make sure we cut out num_bicycle/moped/scooter

Predicting for: num_trips
MSE:  6342512.034754368
[-4.57336044e+02  1.73380941e+02  5.45329988e+00 -7.79674820e+01
  4.45514583e+02  1.26977968e-03 -4.68256904e-04]
Predicting for: mean_distance
MSE:  24093473896.351818
Predicting for: total_duration
MSE:  3221695108734.984


In [19]:
train.drop(["num_trips", "num_bicycle", "num_moped", "num_scooter"], axis=1).head()

,year,month,date_of_month,weekday,average_trip_hour,total_duration,mean_distance
560,2019,11,07,3.989734,13.859992,4565140,39076.406629
541,2019,10,19,6.000000,15.160983,18526387,171754.195796
392,2019,5,23,4.000000,15.070471,10517416,1489.115353
585,2019,12,02,0.978622,14.257210,4219932,26586.637457
630,2020,1,16,3.974426,13.554246,2927874,6007.456943


In [20]:
#xtrain,xtest,ytrain,ytest = train_test_split(x,y,train_size=0.20,random_state=1236)

Catboost


In [21]:
!pip install catboost
from catboost import Pool, CatBoostClassifier, cv, CatBoostRegressor


     |████████████████████████████████| 66.3MB 55kB/s 


In [22]:
model = CatBoostClassifier()

In [23]:
train, test = train_test_split(merge_data, test_size=.30)
#separate data from label
x = train.drop('num_trips',axis=1)
y = train['num_trips']

xtrain,xtest,ytrain,ytest = train_test_split(x,y,train_size=0.20,random_state=1236)
model.fit(xtrain,ytrain)

Learning rate set to 0.072947
0:	learn: 5.2139834	total: 639ms	remaining: 10m 38s
1:	learn: 5.1940827	total: 926ms	remaining: 7m 42s
2:	learn: 5.1713932	total: 1.25s	remaining: 6m 54s
3:	learn: 5.1484389	total: 1.57s	remaining: 6m 31s
4:	learn: 5.1257974	total: 1.89s	remaining: 6m 15s
5:	learn: 5.1042003	total: 2.23s	remaining: 6m 9s
6:	learn: 5.0817824	total: 2.57s	remaining: 6m 4s
7:	learn: 5.0627105	total: 2.87s	remaining: 5m 55s
8:	learn: 5.0410257	total: 3.17s	remaining: 5m 48s
9:	learn: 5.0198118	total: 3.48s	remaining: 5m 44s
10:	learn: 4.9987213	total: 3.78s	remaining: 5m 39s
11:	learn: 4.9853644	total: 4.06s	remaining: 5m 34s
12:	learn: 4.9630922	total: 4.38s	remaining: 5m 32s
13:	learn: 4.9409449	total: 4.68s	remaining: 5m 29s
14:	learn: 4.9189209	total: 4.98s	remaining: 5m 26s
15:	learn: 4.8970885	total: 5.29s	remaining: 5m 25s
16:	learn: 4.8792815	total: 5.58s	remaining: 5m 22s
17:	learn: 4.8568902	total: 5.88s	remaining: 5m 20s
18:	learn: 4.8374006	total: 6.17s	remaining: 

In [24]:
from sklearn import metrics

# make predictions
prediction = model.predict(xtest)

print('Mean Absolute Error:', metrics.mean_absolute_error(ytest, prediction))
print('Mean Squared Error:', metrics.mean_squared_error(ytest, prediction))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest, prediction)))

Mean Absolute Error: 3309.9137931034484
Mean Squared Error: 37000954.24876847
Root Mean Squared Error: 6082.840968558069


XGBoost

In [25]:
import xgboost as xgb

In [26]:
param = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob'}
num_round = 20  # the number of training iterations

In [27]:
from sklearn.datasets import dump_svmlight_file

#separate data from label
x = train.drop(['num_trips', 'date_of_month'],axis=1)
y = train['num_trips']

xtrain,xtest,ytrain,ytest = train_test_split(x,y,train_size=0.20,random_state=1236)


model = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

model.fit(xtrain,ytrain)

prediction = model.predict(xtest)


[20:55:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [28]:
print('Mean Absolute Error:', metrics.mean_absolute_error(ytest, prediction))
print('Mean Squared Error:', metrics.mean_squared_error(ytest, prediction))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest, prediction)))


Mean Absolute Error: 3998.4756327215673
Mean Squared Error: 42182216.57041434
Root Mean Squared Error: 6494.7837970493165


Random Forest

In [29]:
from sklearn.ensemble import RandomForestClassifier
rclf = RandomForestClassifier()

In [30]:
rclf.fit(xtrain,ytrain)

prediction = rclf.predict(xtest)


In [31]:
print('Mean Absolute Error:', metrics.mean_absolute_error(ytest, prediction))
print('Mean Squared Error:', metrics.mean_squared_error(ytest, prediction))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest, prediction)))

Mean Absolute Error: 5202.753694581281
Mean Squared Error: 67575173.43103448
Root Mean Squared Error: 8220.411998862008


Decision Tree Classifier

In [32]:
pip install --upgrade category_encoders

     |████████████████████████████████| 81kB 3.3MB/s 


In [33]:
import category_encoders as ce
encoder = ce.OrdinalEncoder()


dt_train = encoder.fit_transform(xtrain)

dt_test = encoder.transform(xtest)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [34]:
dt_train.head()

,avg_trip_duration,avg_trip_distance,Max_Temperature_F,Avg_Temperature_F,Min_Temperature_F,Max_DewPoint_F,Avg_DewPoint_F,Min_DewPoint_F,Max_Humidity_%,Avg_Humidity_%,Min_Humidity_%,Max_WindSpeed_mph,Avg_WindSpeed_mph,Min_WindSpeed_mph,Max_Pressure_Hg,Avg_Pressure_Hg,Min_Pressure_Hg,Precipitation_inches,year,prev_day_trips,day__Fri,day__Mon,day__Sat,day__Sun,day__Thur,day__Tues,day__Wed,mo__Apr,mo__Aug,mo__Dec,mo__Feb,mo__Jan,mo__Jul,mo__Jun,mo__Mar,mo__May,mo__Nov,mo__Oct,mo__Sept
1180,1016.339836,60922.328340,97,85.5,73,76,71.5,66,93,66.0,37,14,7.8,0,29.3,29.3,29.2,0.0,2020,39,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
924,782.624589,207259.104707,71,63.2,60,65,59.7,57,93,88.5,78,15,10.4,0,29.7,29.6,29.6,0.0,2020,13504,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
798,1137.106312,4751.219269,77,56.3,34,44,35.0,30,89,51.9,20,18,7.1,0,29.6,29.5,29.4,0.0,2020,18,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1426,660.051133,8410.227496,81,60.5,41,55,47.7,38,90,67.5,30,14,5.4,0,29.7,29.6,29.6,0.0,2020,55,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
608,742.091837,3306.234694,65,57.8,53,32,27.2,20,40,31.6,21,23,17.3,12,29.6,29.6,29.4,0.0,2019,8484,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0


In [35]:
from sklearn.tree import DecisionTreeClassifier

In [36]:
model = DecisionTreeClassifier(criterion='MSE', max_depth=3, random_state=0)

#def AccuracyTracker(Xtrain,Xtest,ytrain,ytest,n):
model = DecisionTreeClassifier(max_leaf_nodes=50,random_state=50)

# fit the model
model.fit(dt_train, ytrain)


#for i in range(2,50):
#AccuracyTracker(dt_train,xtest,ytrain,ytest,4)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=50,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=50, splitter='best')

In [37]:
prediction = model.predict(dt_test)

In [38]:
print('Mean Absolute Error:', metrics.mean_absolute_error(ytest, prediction))
print('Mean Squared Error:', metrics.mean_squared_error(ytest, prediction))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest, prediction)))

Mean Absolute Error: 6104.587438423645
Mean Squared Error: 101681458.17610838
Root Mean Squared Error: 10083.722436486854
